# Customizable pytorch LLM training pipeline

## 왜 이 노트북을 사용하나요?

많은 사람들처럼, 저는 LLMs에 대한 기술과 이해를 향상시키려고 노력하고 있습니다.
매일 새로운 것들이 등장하고 있으며, 따라가기가 어렵습니다.
몇 줄의 코드로 LLM을 미세 조정할 수 있는 훌륭한 리소스들이 많이 있습니다 (transformers, trl, auto-train, ...).
하지만 이러한 준비된 라이브러리를 사용할 때는 많이 배우는 느낌이 들지 않습니다.
그래서 저는 최종 사용자에게 모든 세부 사항을 가능한 한 많이 보여주고 사용자 정의할 수 있는 간단한 독립형 노트북을 제안합니다. 물론 바퀴를 다시 발명하지 않고요.

제가 말했듯이, 저는 현재 이 모든 것을 배우고 있으며, 제 파이프라인에 오류나 쉬운 개선점이 있을 수 있습니다. 발견하시면 주저하지 말고 알려주셔서 모두가 개선할 수 있도록 도와주세요!

## 사용자 정의를 위한 아이디어

이 노트북을 넘어서는 몇 가지 아이디어:
- 다른 LLM backbones 시도하기
- 더 복잡한 prediction head 추가하기
- 다양한 loss functions 탐색하기
- 다른 pooling method 시도하기

## 아직 작동하지 않는 것들
이것이 어떻게 작동할 수 있는지 알려주세요!

- float16, bfloat16, 또는 int8, int4로 LLM을 로드하고 훈련하기: 이 작업을 수행하기 위해 파이프라인 내부에서 무엇을 해야 하는지 아직 모르겠습니다 (메모리 필요량 측면에서 모든 것이 바뀔 것입니다).
- 모델을 저장할 때 메모리를 절약하기 위해 peft weights + custom layers만 저장하기

## 탐색할 다음 단계
비슷한 노트북을 탐색하고 공유할 시간을 찾으려고 합니다:
- qlora
- torchtune
- 그 외에 무엇이 있을까요? 댓글로 아이디어를 공유해주세요!

**Happy Kaggling!**

In [1]:
# # install latest libraries
# ! pip install -q /kaggle/input/lal-scoring-wheels/tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --no-deps
# ! pip install -q /kaggle/input/lal-scoring-wheels/transformers-4.40.0-py3-none-any.whl --no-deps
# ! pip install -q /kaggle/input/lal-scoring-wheels/peft-0.10.0-py3-none-any.whl --no-deps
# ! pip install -q /kaggle/input/lal-scoring-wheels/accelerate-0.29.3-py3-none-any.whl --no-deps

In [ ]:
# !pip install tokenizers==0.19.1
# !pip install transformers==4.40.0
# !pip install peft==0.10.0
# !pip install accelerate==0.29.3

In [5]:
import numpy as np
import pandas as pd
import torch
import time
import datetime

In [8]:
import torch
from transformers import AutoModel

# 모델 경로 설정
model_path = "/Users/petersong/code/f-lab/LoRA/models/gemma-transformers-1.1-2b-it-v1"

# 모델 로드
model = AutoModel.from_pretrained(model_path)

# state_dict의 키 출력
for name, param in model.state_dict().items():
    print(name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.25s/it]

embed_tokens.weight
layers.0.self_attn.q_proj.weight
layers.0.self_attn.k_proj.weight
layers.0.self_attn.v_proj.weight
layers.0.self_attn.o_proj.weight
layers.0.mlp.gate_proj.weight
layers.0.mlp.up_proj.weight
layers.0.mlp.down_proj.weight
layers.0.input_layernorm.weight
layers.0.post_attention_layernorm.weight
layers.1.self_attn.q_proj.weight
layers.1.self_attn.k_proj.weight
layers.1.self_attn.v_proj.weight
layers.1.self_attn.o_proj.weight
layers.1.mlp.gate_proj.weight
layers.1.mlp.up_proj.weight
layers.1.mlp.down_proj.weight
layers.1.input_layernorm.weight
layers.1.post_attention_layernorm.weight
layers.2.self_attn.q_proj.weight
layers.2.self_attn.k_proj.weight
layers.2.self_attn.v_proj.weight
layers.2.self_attn.o_proj.weight
layers.2.mlp.gate_proj.weight
layers.2.mlp.up_proj.weight
layers.2.mlp.down_proj.weight
layers.2.input_layernorm.weight
layers.2.post_attention_layernorm.weight
layers.3.self_attn.q_proj.weight
layers.3.self_attn.k_proj.weight
layers.3.self_attn.v_proj.weight
la

# 실험 구성
설정이 완료되면 하이퍼파라미터만 조정하여 최적의 모델을 찾으면 됩니다.

In [1]:
class Config():
    def __init__(self):
        # 문제와 관련된 매개변수
        self.num_classes = 1 # 회귀를 위한 1개의 클래스
        
        # 네트워크와 관련된 매개변수
        self.architecture = {"backbone": "models/gemma-transforemrs-1.1-2b-it-v1",
                             "params": {}}
        self.remove_layers = 8 # 모델을 작게 만들기 위해 제거할 레이어 수
        self.freeze_layers = None # 훈련 매개변수 수를 줄이기 위해 고정할 레이어 수
        self.use_lora = True
        self.lora_config = {"r": 16, # 분해된 행렬의 랭크 (높을수록 메모리 절약이 적음)
                            "lora_alpha": 32, # 스케일링 팩터, https://www.entrypointai.com/blog/lora-fine-tuning/에 따르면 2xr이어야 함
                            "lora_dropout": 0.1, # 일반적인 dropout
                            # 백본에 따라 모듈을 올바르게 이름 지정해야 함
                            # attention blocks의 linear layers를 찾아야 함
                            "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                                               "gate_proj", "up_proj", "down_proj",],
                           }
        
        self.token_info = {"padding" :"longest", # 배치는 가장 긴 시퀀스의 길이가 될 것임
                           "max_length" : 256, # 로컬에서 1024로 훈련, kaggle 노트북 내에서 훈련하기 위해 낮춤
                           "truncation": True,
                           "pad_to_multiple_of" : 512 
                           # 패딩을 특정 값의 배수로 맞추는 옵션. 
                           # VIDIA의 Tensor Cores와 같은 하드웨어 가속기에서는, 연산 속도를 최적화하기 위해 배치 크기나 입력 길이가 특정 배수로 정렬될 때 더 나은 성능을 발휘하는 경우가 많습니다. 
                           # 이때 pad_to_multiple_of를 사용하여 입력을 정렬할 수 있습니다
                          }

        # 훈련과 관련된 매개변수
        self.max_epochs = 1 # 에포크 수

        self.initial_lr =1e-4
        self.optimizer_name = "AdamW" #"AdamW" # 8 bit adam AdamW8bit 시도     
        self.optimizer_params = {"lr": self.initial_lr, 
                                 "weight_decay":1e-2
                                }
        self.loss_config = {"loss_name" : "MSELoss",
                            "reduction":"mean",
                           }
        # OneCycleLR는 학습률을 한 사이클 동안 증가시켰다가 감소시키는 방식으로 조정하는 스케줄러
        # 학습률을 효과적으로 조정하여 모델의 수렴 속도를 높이고, 과적합을 방지하는 데 도움을 줌
        # OneCycleLR 스케줄러는 특히 빠른 수렴과 일반화 성능을 개선하는 데 유용한 것으로 알려져 있음
        self.scheduler_name = "OneCycleLR"
        self.steps_per_epochs = -1 # 자동으로 덮어씌워짐
        self.scheduler_params={
                              #학습률의 최대값을 설정합니다. 이는 optimizer_params에서 정의된 학습률을 사용
                              "max_lr":self.optimizer_params["lr"] if type(self.optimizer_params)==dict else self.optimizer_params[-1]["lr"],
                               #초기 학습률을 max_lr / div_factor로 나눕니다. 초기에는 작은 학습률로 시작하여 안정적으로 모델을 학습시키고, 이후 학습률을 증가시키는 전략
                               "div_factor":10,
                              "steps_per_epoch": self.steps_per_epochs,
                              #학습이 완료될 때 최종 학습률을 max_lr / final_div_factor로 나눕니다. 이는 학습률을 최종적으로 조정하는 데 사용됩니다.
                              "final_div_factor":1e2, #1e2
                              #학습률 조정 전략을 설정합니다. "cos"는 코사인 곡선을 사용하여 학습률을 조정합니다.
                              "anneal_strategy":"cos", #"cos"
                              #세 단계 학습률 조정을 사용할지 여부를 설정합니다. 세 단계 학습률 조정은 학습률을 세 단계로 나누어 조정합니다.
                               "three_phase" : False,
                              #학습률 조정이 시작되는 지점을 설정합니다. 이 값은 학습률 조정이 시작되는 지점을 나타냅니다. 
                              #True로 설정하면: 학습의 마지막 단계에서 학습률을 극도로 낮게 조정하여 모델이 안정적으로 최종 가중치에 수렴할 수 있음
                              "pct_start":0.1, # 기본값 0.3으로 전체 학습 중 학습률이 증가하는 단계가 어느 시점에서 시작할지를 설정합니다. 0.1로 설정되면, 학습의 10%에서 학습률이 최대에 도달하게 됩
                              #전체 학습 에포크 수를 설정합니다. max_epochs의 값을 사용
                              "epochs": self.max_epochs}
        '''
        학습률 스케줄링의 변화 예시
        **max_lr=1e-2, div_factor=20, final_div_factor=1e3로 설정하면:
        초기 학습률이 1e-2 / 20 = 5e-4로 매우 낮고, 학습 후반부에서는 1e-2 / 1e3 = 1e-5로 수렴하게 됩니다. 이는 매우 천천히 학습이 이루어지는 방식이며, 모델이 안정적으로 수렴할 가능성이 높습니다.
        **max_lr=1e-1, div_factor=5, final_div_factor=1e2로 설정하면:
      초기 학습률이 2e-2로 상당히 크고, 최종 학습률도 1e-3로 높습니다. 이는 빠른 학습을 목표로 하며, 불안정할 수 있지만 최적화 속도는 빠를 수 있습니다.
        '''
        
        self.eval_on_train = False # 과적합을 모니터링하기 위해 훈련 세트에서 정확한 메트릭을 계산하고 싶을 수 있음
        self.batch_size = 1 # 작게 시작합시다
        self.gradient_accumulation = 16 // self.batch_size # 낮은 배치 크기로 훈련하지만 몇 개의 샘플에서 그래디언트를 계산할 수 있음
        self.mixed_precision = True
        self.num_workers = 2 # 데이터 로더에서 병렬 처리를 위해 2개의 작업자 사용
        self.pin_memory = True # GPU에 데이터를 미리 로드하여 데이터 전송 속도를 높임
        self.clip_value = 10.0

        # 로그와 관련된 매개변수
        self.verbose = 1 # 훈련 도중 얼마나 자주 로그를 출력할 것인지. 0일 경우 로그 출력 안함
        self.save_path = "working/" # 훈련 중 저장할 

# 학습과 테스트를 위한 데이터셋 경로 설정
PATH_TO_DATA = "input/learning-agency-lab-automated-essay-scoring-2"
exp_config = Config()

# Define datasets

In [ ]:
from dataclasses import dataclass
from torch.utils.data import DataLoader, Dataset
from typing import Optional, Union, Any
from transformers import DataCollatorWithPadding

from transformers import AutoTokenizer

def define_tokenizer(cfg):
    """
    기본 AutoTokenizer 사용
    """
    tokenizer = AutoTokenizer.from_pretrained(cfg.architecture["backbone"])    
    return tokenizer
    
class LALDataset(Dataset):
    """
    LALDataset 클래스: PyTorch Dataset을 상속받아 데이터를 모델에 맞게 준비하는 클래스를 정의합니다. 
    각 데이터셋의 정의는 학습 및 추론에서 어떻게 데이터를 사용해야 할지 결정하는 중요한 부분입니다.
    """
    def __init__(self, df, config, inference, remove=True):
        """
        df: pandas 데이터프레임
        config: 학습 및 토큰화 관련된 설정 값
        inference (bool): 추론 모드인지 여부를 나타냄
        remove (bool): 학습에 필요하지 않은 열을 제거할지 여부를 결정
        """
        self.df = df
        # tokenizer는 datacollator에서 사용되므로 정의되어야 함
        self.tokenizer = define_tokenizer(config)
        self.inference = inference
        self.config = config
        self.remove = remove
        
    def __len__(self):
        """
        데이터셋의 전체 크기를 반환
        """
        return len(self.df)
    
    def __getitem__(self, row_idx):
        
        full_text = self.df.loc[row_idx, "full_text"]
        # self.tokenizer()를 사용해 해당 텍스트를 토큰화하고, 주어진 max_length와 truncation 설정에 맞춰 조정
        tokenized_text = self.tokenizer(full_text,
                                        return_offsets_mapping=False, # 주로 엔티티 인식을 위해 필요
                                        truncation=self.config.token_info["truncation"],
                                        max_length=self.config.token_info["max_length"])
        
        labels = self.df.loc[row_idx, "score"]
        # df에서 해당 행의 점수(레이블)를 가져와 labels로 저장
        # 여기서 eos 토큰을 끝에 추가하여 CLS 토큰으로 작동하게 함
        # 모델이 인과적 Attention을 가지는 GPT 계열 모델이므로, 입력 텍스트의 마지막에 eos 토큰을 추가하여 CLS 역할을 하게 합니다
        tokenized_text.input_ids.append(self.tokenizer.eos_token_id)
        tokenized_text.attention_mask.append(1)
        """
        토큰화: 입력 문장이 주어지면 토크나이저를 사용해 input_ids 리스트로 변환됩니다. 예를 들어, "Hello, world!"가 [100, 200, 300]으로 변환됩니다.
        EOS 토큰 추가: 토크나이저에서 미리 정의된 EOS 토큰(eos_token_id)을 input_ids 리스트에 추가합니다. 예를 들어, [100, 200, 300] -> [100, 200, 300, 500].
        Attention Mask 확장: attention_mask는 각 토큰에 대해 1로 설정하여 모델이 모든 토큰을 학습하도록 합니다. 여기서 EOS 토큰을 추가했으므로, 
        이에 대한 Attention Mask도 1을 추가하여 [1, 1, 1] -> [1, 1, 1, 1]로 확장됩니다.
        """

        out_dict = {
                "input_ids": tokenized_text.input_ids,
                "attention_mask": tokenized_text.attention_mask,
                "labels": torch.Tensor([labels])
            }
        return out_dict


def define_loader(dataset, config, inference):
    """
    훈련 및 테스트를 위한 dataloader 생성
    """
    num_workers = config.num_workers
    pin_memory = config.pin_memory
    
    # collate_fn = None
    # data collator를 사용해 토큰화된 데이터를 패딩하는 방법을 사용
    """
    DataCollatorWithPadding는 토큰화된 데이터를 동일한 길이로 맞춰주는 역할을 합니다. 
    이는 특히 배치(batch) 처리를 할 때 중요합니다. 
    왜냐하면, NLP 모델의 입력은 일반적으로 고정된 길이여야 하고, 
    텍스트 데이터는 길이가 다양하기 때문에 짧은 텍스트들은 패딩을 통해 길이를 맞춰줘야 합니다
    """
    collate_fn = DataCollatorWithPadding(tokenizer=dataset.tokenizer,
                                         padding=config.token_info["padding"],
                                         max_length=config.token_info["max_length"],
                                         pad_to_multiple_of=config.token_info["pad_to_multiple_of"]
                                    )
    #padding이 True일 경우 모든 배치에서 가장 긴 문장 또는 옵션에 의해 결정되나
    #longest는 배치 내에서 긴 문장에 따라 적용되어 각 문장에 불필요한 패딩을 최소화 할 수 있음

    # Dataset을 받아 DataLoader로 변환
    loader = DataLoader(
                dataset,
                batch_size=config.batch_size,
                shuffle=not inference,
                drop_last=not inference,
                # 데이터 로드를 위한 병렬 처리 워커 수
                num_workers=num_workers,
                # GPU에 데이터를 미리 로드하여 데이터 전송 속도를 높임
                pin_memory=pin_memory, 
                collate_fn=collate_fn,
                # worker_init_fn=worker_init_fn,
            )
    return loader


def get_dataset_and_loader(df, config, inference, remove=True):
    """
    LALDataset 객체 생성: 주어진 데이터프레임과 설정을 사용해 데이터셋을 만듭니다.
    DataLoader 생성: 만들어진 데이터셋을 기반으로 데이터 로더를 반환합니다. 
    학습 및 추론에서 사용되는 데이터셋과 데이터 로더를 모두 반환
    """
    dataset = LALDataset(df, config, inference, remove=remove)
    loader = define_loader(dataset, config, inference)
    return dataset, loader

def create_loaders(df, train_idx, valid_idx, config, eval_on_train):
    """
    주어진 학습 및 검증 데이터 인덱스(train_idx, valid_idx)에 따라 학습과 검증용 데이터셋을 나누고, 각각의 데이터 로더를 생성합니다.
    eval_on_train 옵션**: 만약 eval_on_train이 True일 경우, 학습 데이터도 검증용으로 사용할 수 있도록 추가적으로 검증용 데이터 로더(train_aux_dl)를 생성합니다.
    최종적으로 학습용(train_dl)과 검증용(valid_dl) 데이터 로더를 반환하고, 평가할 데이터 로더와 이름(eval_loaders, eval_names)도 함께 반환합니다.
    """
    # 추론을 위해 더 큰 max length 설정 가능
    valid_config = copy.deepcopy(config)
    valid_config.token_info['max_length'] = config.token_info['max_length']
    
    _, train_dl = get_dataset_and_loader(df=df.iloc[train_idx].reset_index(drop=True),
                                        config=config,
                                        inference=False,
                                        )

    _, valid_dl = get_dataset_and_loader(df=df.iloc[valid_idx].reset_index(drop=True),
                                        config=valid_config,
                                        inference=True)

    if eval_on_train:
        _, train_aux_dl = get_dataset_and_loader(df=df.iloc[train_idx].reset_index(drop=True),
                                                config=valid_config,
                                                inference=True)

        eval_loaders = [train_aux_dl, valid_dl]
        eval_names = ["train", "valid"]
    else:
        eval_loaders = [valid_dl]
        eval_names = ["valid"]
    return train_dl, valid_dl, eval_loaders, eval_names

# 네트워크의 아키텍처 정의

여기서는 LORA로 미세 조정된 LLM backbone과 linear head로 구성된 간단한 아키텍처를 사용합니다.
우리는 최종 점수 예측을 위해 마지막 eos_token만 사용합니다.

torch.nn.Module을 사용하여 이 아키텍처를 쉽게 사용자 정의할 수 있습니다:
- tf-idf, num_words, engineered features 등과 같은 메타데이터를 입력으로 추가
- 최종 head를 더 복잡하게 만들기 (MLP with activations 등)
- eos_token pooling 대신 pooling methods 시도 등

In [5]:
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoConfig

class CustomLLM(torch.nn.Module):
    """
    사용자 정의 아키텍처를 설정할 수 있는 클래스입니다.
    """
    def __init__(self, cfg, eos_token_id):
        super().__init__()
        self.num_classes = cfg.num_classes  # 예측할 클래스의 수
        self.eos_token_id = eos_token_id  # 문장의 끝을 나타내는 토큰 ID
        self.model_config = AutoConfig.from_pretrained(
                cfg.architecture["backbone"],
            )  # 사전 학습된 모델의 구성 불러오기

        self.activation = torch.nn.Identity()  # 기본 활성화 함수 설정
        # 네트워크의 backbone 설정
        self.backbone  = AutoModelForCausalLM.from_pretrained(cfg.architecture["backbone"],
                                                                    device_map="cpu",  # 모델을 로드할 장치
                                                                    load_in_4bit=False,  # 4비트 로드 설정
                                                                    torch_dtype=torch.float32,  # 데이터 타입 설정
                                                                    **cfg.architecture["params"])
        # 사용자 정의 head를 사용하기 위해 기존 head 제거
        self.backbone.lm_head = torch.nn.Identity()

        if cfg.remove_layers is not None:
            # 마지막 레이어 제거: 불필요한 레이어 제거
            self.backbone.layers = self.backbone.model.layers[:-cfg.remove_layers]          
        
        if hasattr(cfg, "num_layers_to_freeze"):
            print(f"freezing {cfg.num_layers_to_freez} layers.")
            if cfg.num_layers_to_freeze > 0:
                if cfg.freeze_embeddings:
                    # 임베딩 레이어 고정
                    for param in self.backbone.embed_tokens.parameters():
                        param.requires_grad = False
                # 첫 번째 레이어 고정: 나머지 마지막 레이어만 훈련
                for layer in self.backbone.model.layers[:cfg.num_layers_to_freeze]:
                    for param in layer.parameters():
                        param.requires_grad = False
                
        if cfg.use_lora:
            # peft 라이브러리에서 lora 적용
            peft_config = LoraConfig(
                task_type=TaskType.CAUSAL_LM,
                inference_mode=False,  # 추론 모드 설정
                r=cfg.lora_config["r"],
                lora_alpha=cfg.lora_config["lora_alpha"],
                lora_dropout=cfg.lora_config["lora_dropout"],
                target_modules=cfg.lora_config["target_modules"],
            )
            
            self.backbone = get_peft_model(self.backbone, peft_config)
        else:
            print("NOT USING LORA")
            
        # gradient checkpoint는 나중에 사용
        # self.transformers_model.gradient_checkpointing_enable()
                    
        self.final_linear = torch.nn.Linear(self.model_config.hidden_size, cfg.num_classes)  # 최종 예측을 위한 선형 레이어
        
        if cfg.use_lora:
            self.backbone.print_trainable_parameters()  # 훈련 가능한 파라미터 출력
        
    def forward(self, batch):
        x = batch["input_ids"]  # 입력 데이터의 ID
        # 각 예제에 하나의 eos_token만 있다고 가정
        eos_positions = torch.argwhere(x == self.eos_token_id)[:, 1]
        x = self.backbone(
            input_ids=x,
            attention_mask=batch["attention_mask"],
        )["logits"]  # 모델의 출력 logits
        
        # eos_token에 해당하는 위치의 logits만 사용
        x = x[torch.arange(x.shape[0]), eos_positions]  # (bs, hidden_size)
        
        logits = self.final_linear(x)  # 최종 예측 생성

        return {"logits": logits}  # 예측 결과 반환

# Training recipe

You may need to change this if you make significant changes in your modelling apporach

In [6]:
from dataclasses import dataclass
from typing import List, Any, Dict
from torch.nn.utils import clip_grad_norm_
from abc import abstractmethod
from sklearn.base import BaseEstimator
import json
from pathlib import Path
from tqdm.notebook import tqdm
import copy

# AdamW 옵티마이저에서 weight decay를 적용하지 않을 레이어들
ALL_LAYERNORM_LAYERS = [torch.nn.LayerNorm, torch.nn.Embedding]

def get_parameter_names(network, forbidden_layer_types):
    """
    모델 파라미터 중 금지된 레이어에 속하지 않는 파라미터의 이름을 반환합니다.
    """
    result = []
    for name, child in network.named_children():
        result += [
            f"{name}.{n}"
            for n in get_parameter_names(child, forbidden_layer_types)
            if not isinstance(child, tuple(forbidden_layer_types))
        ]
    # 모델에 특정된 파라미터 추가 (nn.Parameter로 정의된 것들)
    result += list(network._parameters.keys())
    return result

def define_loss_function(loss_config):
    """
    기본 torch 손실 함수 또는 로컬에서 정의된 손실 함수
    """
    copy_config = copy.copy(loss_config)
    loss_name = copy_config.pop('loss_name')
    try:
        loss_fn = getattr(torch.nn, loss_name)(**copy_config)
    except AttributeError:
        try:
            loss_fn = globals().get(loss_name)(copy_config)
        except:
            raise NotImplementedError("알 수 없는 손실 함수 :", loss_name)
    return loss_fn

def prepare_log_folder(log_path):
    """
    실험 폴더를 생성하는 유틸리티 함수
    로그를 저장할 디렉토리를 생성합니다.
    로그는 log_path/date_of_day/exp_id에 저장됩니다.

    Args:
        log_path (str): 디렉토리 경로

    Returns:
        str: 생성된 로그 폴더의 경로
    """
    today = str(datetime.date.today())
    log_today = os.path.join(log_path, today)

    if not os.path.exists(log_today):
        Path(log_today).mkdir(parents=True)

    exp_id = (
        np.max([int(f) if str(f).isdigit() else -1 for f in os.listdir(log_today)]) + 1
        if len(os.listdir(log_today))
        else 0
    )
    log_folder = os.path.join(log_today, f"{exp_id}")

    assert not os.path.exists(log_folder), "실험이 이미 존재합니다"
    os.mkdir(log_folder)
    print("로그를 저장합니다 :", log_folder)
    return log_folder

def save_config(config, folder):
    """
    설정을 json으로 저장하고, 데이터 및 모델 설정을 복사합니다.

    Args:
        config (Config): 설정 객체
        folder (str): 저장할 폴더 경로
    """
    with open(os.path.join(folder, "config.json"), "w") as f:
        json.dump(config.__dict__.copy(), f)

@dataclass
class AbstractBaseModel(BaseEstimator):
    """ scikit과 유사한 모델을 위한 추상 클래스.
        훈련, 추론, 저장, 로드 등을 구축할 수 있습니다.
    """

    network: torch.nn.Module = None
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    seed: int = 42
    mixed_precision: bool = False

    def __post_init__(self):
        torch.manual_seed(self.seed)
        self.network.to(self.device)

    def fit(
        self,
        train_dataloader,
        eval_loaders=None,
        eval_names=None,
        eval_metric=None,
        loss_config=None,
        max_epochs=100,
        callbacks=None,
        optimizer_name="Adam",
        optimizer_params={"lr": 1e-3},
        gradient_accumulation=None,
        scheduler_name=None,
        scheduler_params=None,
        mixed_precision=False,
        clip_value=None,
        log_path=None,
        verbose=1,
    ):
        """
        self.network에 저장된 신경망을 훈련합니다.
        train_dataloader를 사용하여 훈련 데이터를 사용하고
        eval_loaders를 사용하여 검증합니다.

        Parameters
        ----------
        train_dataloader : Dataloader
            훈련 세트
        eval_loader : list of dataloaders
            마지막 하나는 조기 중단에 사용됩니다.
        eval_name : list of str
            평가 세트 이름 목록
        eval_metric : list of str
            평가 메트릭 목록
            마지막 메트릭은 조기 중단에 사용됩니다.
        loss_name : Name
            PyTorch 손실 함수 이름
        max_epochs : int
            훈련 중 최대 에포크 수
        num_workers : int
            torch.utils.data.DataLoader에서 사용되는 작업자 수
        drop_last : bool
            훈련 중 마지막 배치를 버릴지 여부
        pin_memory: bool
            훈련 중 pin_memory를 True 또는 False로 설정할지 여부
        from_unsupervised: unsupervised trained model
            이전에 self supervised 모델을 시작 가중치로 사용
        clip_value: float (기본값 None)
            그래디언트 클리핑
        """
        # 모델 이름 업데이트

        self.max_epochs = max_epochs
        self._stop_training = False
        self.optimizer_name = optimizer_name
        self.optimizer_params = optimizer_params
        eval_loaders = eval_loaders if eval_loaders else []       
        self.mixed_precision = mixed_precision
        self.clip_value = clip_value
        self.verbose = verbose
        self.gradient_accumulation = gradient_accumulation
        self.metrics = eval_metric
        
        if loss_config is None:
            raise(NotImplementedError, "손실을 지정하세요")
        else:
            self.loss_fn = define_loss_function(loss_config)
        
        self._set_optimizer()
        
        # 스케줄러 설정
        self.scheduler_fn = getattr(torch.optim.lr_scheduler, scheduler_name)  # torch 스케줄러만 허용
        self.scheduler_params = copy.copy(scheduler_params)
        self.scheduler = self.scheduler_fn(self._optimizer, **self.scheduler_params)
        
        # 에포크 동안 훈련 루프
        start_time = time.time()
        for epoch_idx in range(self.max_epochs):
            self.epoch_idx = epoch_idx
            epoch_loss, epoch_lr = self._train_epoch(train_dataloader)
            msg = f"epoch {epoch_idx:<3} | lr: {epoch_lr:.2e} | loss: {epoch_loss:.4f} "
            # 모든 평가 세트에 대해 예측 에포크 적용
            if ((self.verbose != 0) and (epoch_idx % self.verbose == 0)) or (epoch_idx==self.max_epochs-1):
                for eval_name, valid_dataloader in zip(eval_names, eval_loaders):
                    with torch.no_grad():
                        prob_pred, prob_true, scores = self._predict_epoch(eval_name, valid_dataloader)
                    for metric_name, metric_score in scores:
                        msg += f"| {metric_name:<3} ({eval_name}): {metric_score:.4f} "
            total_time = int(time.time() - start_time)
            msg += f"|  {str(datetime.timedelta(seconds=total_time)) + 's':<6}"
            print(msg)
        print("훈련 종료!")
        self.network.eval()
        return prob_pred, prob_true
        
    def predict_proba(self, dataloader, return_target=False):
        """
        배치에 대한 예측 수행 (유효성 검사)
        Parameters
        ----------
        X : a :tensor: `torch.Tensor`
            입력 데이터
        Returns
        -------
        predictions : np.array
            회귀 문제의 예측
        """
        self.network.eval()
        results_prob = []
        results_targets = []
        pbar = tqdm(dataloader,
                     leave=False,
                     total=len(dataloader),
                     desc=f'Inference')
        
        with torch.no_grad():
            for batch in pbar:
                out_probs = self._predict_batch(batch).cpu()
                results_prob.append(out_probs)

                if return_target:
                    targets = batch["labels"]
                    targets = targets.to("cpu").detach()
                    results_targets.append(targets)

        res_prob = self.stack_preds(results_prob)                
        if return_target:
            res_target = self.stack_targets(results_targets)
            return res_prob, res_target
        else:
            return res_prob

    def save_model(self, path, model_name):
        """
        모델을 저장합니다.

        사용자는 경로와 모델 이름을 모두 지정할 수 있습니다.
        모델 이름이 주어지지 않으면 자동으로 생성됩니다.
        """
        Path(path).mkdir(parents=True, exist_ok=True)
        # 추론 중 GPU 사용량을 줄이기 위해 절반 정밀도로 state_dict 저장: 혼합 정밀도로 훈련된 경우 동일한 결과
        torch.save(self.network.half().state_dict(), Path(path).joinpath(f"{model_name}.pt"))
        # torch.save(self.network.state_dict(), Path(path).joinpath(f"{model_name}.pt"))
        return

    def _train_epoch(self, train_loader):
        """
        self.network에서 네트워크의 한 에포크를 훈련합니다.
        Parameters
        ----------
        train_loader : a :class: `torch.utils.data.Dataloader`
            훈련 세트가 있는 DataLoader
        """
        self.network.train()
        num_iter_epoch = len(train_loader)
        pbar = tqdm(enumerate(train_loader),
                                     leave=False,
                                     total=len(train_loader),
                                     desc=f'train epoch {self.epoch_idx}')
        
        epoch_loss = 0
        for batch_idx, batch in pbar:
            batch_loss = self._train_batch(batch, batch_idx, num_iter_epoch)
            epoch_loss = (train_loader.batch_size*batch_idx*epoch_loss + train_loader.batch_size*batch_loss) / (train_loader.batch_size*(batch_idx+1))            
            pbar.set_description(f'train epoch {self.epoch_idx}: loss {epoch_loss:.3f}', refresh=True)
            # 스케줄러 업데이트
            self.scheduler.step()

        epoch_lr = self._optimizer.param_groups[-1]["lr"]
        return epoch_loss, epoch_lr

    def _train_batch(self, batch, batch_idx, num_iter_epoch):
        """
        데이터의 한 배치를 훈련합니다.
        Parameters
        ----------
        batch_logs : dict
            "batch_size"와 "loss"가 있는 사전
        """
        self._send_batch_to_device(batch)
                                   
        with torch.cuda.amp.autocast(enabled=self.mixed_precision):
            # float16 훈련을 위한 혼합 정밀도 사용
            y = batch["labels"]
            batch_logs = {"batch_size": y.shape[0]}
            out_probs = self.network(batch)
            # 그래디언트 누적에 의한 손실 계산
            loss = self.loss_fn(out_probs["logits"], y.unsqueeze(-1)) / self.gradient_accumulation
            self.scaler.scale(loss).backward()
            
        if ((batch_idx + 1) % self.gradient_accumulation == 0) or ((batch_idx + 1)==num_iter_epoch):
            # 역전파 및 최적화 수행
            if self.clip_value is not None:
                self.scaler.unscale_(self._optimizer)
                clip_grad_norm_(self.network.parameters(), max_norm=self.clip_value)

            self.scaler.step(self._optimizer)
            self.scaler.update()
            # step을 호출할 때만 그래디언트를 0으로 설정
            self._optimizer.zero_grad(set_to_none=True)
        return loss.detach().item()

    def _predict_epoch(self, name, loader):
        """
        에포크를 예측하고 메트릭을 업데이트합니다.
        Parameters
        ----------
        name : str
            검증 세트의 이름
        loader : torch.utils.data.Dataloader
                검증 세트가 있는 DataLoader
        """
        prob_pred, prob_true = self.predict_proba(loader, return_target=True)
        
        scores = []
        for metric_fn in self.metrics:
            metric_score = metric_fn(prob_true, prob_pred)
            scores.append((metric_fn._name, metric_score))
        # 여기서 메트릭을 계산해야 함
        return prob_pred, prob_true, scores

    def stack_preds(self, list_prob):
        return torch.vstack(list_prob)

    def stack_targets(self, list_prob):
        return torch.hstack(list_prob)

    def _send_batch_to_device(self, batch):
        for key, value in batch.items():
            batch[key] = value.to(self.device)
            
    def _predict_batch(self, batch):
        """
        데이터의 한 배치를 예측합니다.
        """
        with torch.cuda.amp.autocast(enabled=self.mixed_precision):
            self._send_batch_to_device(batch)
            # 모델 출력 계산
            out_probs = self.network(batch)["logits"]
            # 활성화 적용
            if isinstance(self.network, torch.nn.DataParallel):
                # 데이터 병렬 처리
                out_probs = self.network.module.activation(out_probs)
            else:
                out_probs = self.network.activation(out_probs)
            
        return out_probs.detach()

    def _set_optimizer(self):
        """옵티마이저 설정."""
        
        name = self.optimizer_name

        # 레이어 정규화에 대해 decay 비활성화
        decay_parameters = get_parameter_names(self.network, ALL_LAYERNORM_LAYERS)
        decay_parameters = [name for name in decay_parameters if "bias" not in name]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in self.network.named_parameters() if (n in decay_parameters and p.requires_grad)
                ],
                "weight_decay": self.optimizer_params["weight_decay"],
            },
            {
                "params": [
                    p for n, p in self.network.named_parameters() if (n not in decay_parameters and p.requires_grad)
                ],
                "weight_decay": 0.0,
            },
        ]
        other_params = self.optimizer_params.copy()
        _ = other_params.pop("weight_decay")
                
        self._optimizer = getattr(torch.optim, name)(optimizer_grouped_parameters, **other_params)        
        self.scaler = torch.cuda.amp.GradScaler(enabled=self.mixed_precision)
        return

# Metrics to track

Here you can define metrics you want to track during model training (every epoch)

In [7]:
from sklearn.metrics import (
    mean_squared_error
)

class RMSE:
    """
    Root Mean Squared Error (RMSE)를 계산하는 클래스입니다.
    """

    def __init__(self):
        self._name = "rmse"  # 메트릭의 이름을 설정합니다.

    def __call__(self, y_true, y_score):
        """
        예측의 평균 제곱 오차(MSE)를 계산합니다.

        Parameters
        ----------
        y_true : np.ndarray
            실제 값의 행렬 또는 벡터
        y_score : np.ndarray
            예측 값의 행렬 또는 벡터

        Returns
        -------
        float
            예측과 실제 값 간의 MSE.
        """
        # squared=False로 설정하여 RMSE를 반환합니다.
        return mean_squared_error(y_true.numpy(), y_score.numpy(), squared=False)

import numpy as np
from numba import jit 

# @jit
def qwk6(a1, a2, max_rat=6):
    """
    CPMP에서 적응된 경쟁 메트릭: https://www.kaggle.com/c/prostate-cancer-grade-assessment/discussion/145105
    """
    assert(len(a1) == len(a2))  # 두 배열의 길이가 같아야 합니다.
    
    a1 = a1.astype(np.int64).reshape(-1)  # a1을 정수형으로 변환하고 1차원으로 변형합니다.
    # 연속적인 예측을 가장 가까운 정수로 변환합니다.
    a2 = np.rint(a2).astype(np.int64).reshape(-1)

    hist1 = np.zeros((max_rat + 1, ))  # a1의 히스토그램을 저장할 배열
    hist2 = np.zeros((max_rat + 1, ))  # a2의 히스토그램을 저장할 배열

    o = 0  # 관측된 차이의 제곱합
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0  # 기대 차이의 제곱합
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]  # 기대 차이의 평균
    return (1 - o / e)  # QWK 점수 반환

class QWK:
    def __init__(self):
        self._name = "qwk"  # 메트릭의 이름을 설정합니다.
    def __call__(self, y_true, y_pred, max_rat=6):
        # QWK 점수를 계산합니다.
        return qwk6(y_true.numpy(), y_pred.numpy())

# from sklearn.metrics import cohen_kappa_score

# class ScikitQWK:
#     """
#     scikit을 사용한 경쟁 메트릭
#     """

#     def __init__(self):
#         self._name = "scikit_qwk"  # 메트릭의 이름을 설정합니다.
#     def __call__(self, y_true, y_pred):
#         y_pred = np.rint(y_pred)  # 예측을 가장 가까운 정수로 변환합니다.
#         return cohen_kappa_score(y_true, y_pred, weights="quadratic")  # QWK 점수 반환

# Puting everything together for training one fold

This is just a simple function that will allow you to train one fold and save the corresponding configs and model checkpoint.

In [3]:
def update_sched_params(config, train_loader):
    """
    이 헬퍼 함수는 에포크당 단계 수를 동적으로 정의할 수 있게 합니다.

    Parameters
    ----------
    - config : 실험 설정
    - train_loader : 이 폴드에 사용되는 훈련 데이터 로더
    """
    nb_epochs = config.max_epochs  # 최대 에포크 수
    is_per_epoch = config.scheduler_params.get("steps_per_epoch", None)

    if is_per_epoch is not None:
        if is_per_epoch <= 0:
            # 자동으로 단계 수를 설정
            config.scheduler_params["steps_per_epoch"] = len(train_loader)
        # 그렇지 않으면 정의된 값을 사용
    
    # get_cosine_schedule_with_warmup을 위한 설정
    warmup_ratio = config.scheduler_params.pop("warmup_ratio", None)

    if warmup_ratio is not None:
        num_train_steps = int(len(train_loader) * nb_epochs)  # 총 훈련 단계 수
        num_warmup_steps = int(num_train_steps * warmup_ratio)  # 워밍업 단계 수
        config.scheduler_params["num_warmup_steps"] = num_warmup_steps
        config.scheduler_params["num_training_steps"] = num_train_steps
        # 그렇지 않으면 정의된 값을 사용
    return config

def train_fold(df,
               train_idx,
               valid_idx,
               config,
               fold_nb):
    """
    주어진 데이터 프레임과 인덱스를 사용하여 하나의 폴드를 훈련합니다.

    Parameters
    ----------
    - df : 데이터 프레임
    - train_idx : 훈련 데이터 인덱스
    - valid_idx : 검증 데이터 인덱스
    - config : 실험 설정
    - fold_nb : 폴드 번호
    """
    print("Num train and valid samples:", train_idx.shape[0], valid_idx.shape[0])
    config = copy.deepcopy(config)  # 설정을 깊은 복사하여 수정
    train_dl, valid_dl, eval_loaders, eval_names = create_loaders(df,
                                                                  train_idx,
                                                                  valid_idx,
                                                                  config,
                                                                  eval_on_train=config.eval_on_train
                                                                  )
    log_folder = prepare_log_folder(config.save_path)  # 로그 폴더 준비
    # eos_token_id를 설정에 추가
    config.eos_token_id = train_dl.dataset.tokenizer.eos_token_id
    save_config(config, log_folder)  # 설정을 저장

    # 네트워크와 모델 초기화
    network = CustomLLM(config, train_dl.dataset.tokenizer.eos_token_id)
    model = AbstractBaseModel(network=network)
        
    # 스케줄러 업데이트
    config = update_sched_params(config, train_dl)

    # 모델 훈련
    prob_pred, prob_true = model.fit(train_dl,
                                      eval_loaders=eval_loaders,
                                      eval_names=eval_names,
                                      eval_metric=[RMSE(), QWK()],  # 평가 메트릭 설정
                                      loss_config=config.loss_config, 
                                      max_epochs=config.max_epochs,
                                      callbacks=None,
                                      optimizer_name=config.optimizer_name,
                                      optimizer_params=config.optimizer_params,
                                      scheduler_name=config.scheduler_name,
                                      scheduler_params=config.scheduler_params,
                                      gradient_accumulation=config.gradient_accumulation,
                                      mixed_precision=config.mixed_precision,
                                      clip_value=config.clip_value,
                                      verbose=config.verbose,
             )

    # 모델 저장
    model.save_model(path=log_folder, model_name=f"fold_{fold_nb}")
    torch.cuda.empty_cache()  # 캐시 비우기
        
    return prob_pred, prob_true  # 예측 결과 반환

# Training: 5 fold cross validation

In [ ]:
# Stratified K-Fold를 사용하여 데이터셋을 나눕니다.
from sklearn.model_selection import StratifiedKFold
import os

# 훈련 데이터를 다운로드합니다.
df_train = pd.read_csv(os.path.join(PATH_TO_DATA, "train.csv"))

# 훈련 및 추론 모드 설정
TRAIN = False  # 훈련을 위해 True로 설정하고, 추론을 위해 False로 설정
INFERENCE = True
DEBUG = False

if TRAIN:
    if DEBUG:
        df_train = df_train[:50]  # 디버그 모드에서는 데이터의 일부만 사용
    # Stratified K-Fold를 사용하여 데이터를 5개의 폴드로 나눕니다.
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # 각 폴드에 대해 훈련 및 검증 인덱스를 생성합니다.
    for fold_nb, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train.score)):
        # 각 폴드에 대해 모델을 훈련합니다.
        prob_pred, prob_true = train_fold(df_train,
                                           train_idx,
                                           valid_idx,
                                           exp_config,
                                           fold_nb=fold_nb,
                                           )
        break  # 첫 번째 폴드만 훈련하고 종료

In [10]:
# Example of what you should see (trained on my personal setup)

# Num train and valid samples: 13845 3462
# Saving logs at : ../../logs/essay_scoring/2024-04-23/2
# trainable params: 9,805,824 || all params: 2,515,978,240 || trainable%: 0.3897420034920493
# epoch 0   | lr: 5.87e-05 | loss: 0.0662 | rmse (valid): 0.5429 | qwk (valid): 0.8105 |  0:45:41s
# epoch 1   | lr: 1.00e-07 | loss: 0.0301 | rmse (valid): 0.5167 | qwk (valid): 0.8358 |  1:31:20s
# End of training!

# Inference

Here is where you can perform simple inference from a previously trained checkpoint.

In [11]:
class SavedConfig:
    """
    저장된 json 파일에서 설정을 로드하기 위한 플레이스홀더 클래스입니다.
    """
    def __init__(self, dic):
        # 딕셔너리의 각 항목을 객체의 속성으로 설정합니다.
        for k, v in dic.items():
            setattr(self, k, v)

def load_model(path, model_name, override_backbone=None):
    """
    이전에 훈련된 모델을 로드합니다.

    Parameters
    ----------
    path : str
        모델이 저장된 경로
    model_name : str
        로드할 모델의 이름
    override_backbone : str, optional
        백본을 덮어쓸 경우 사용할 백본의 경로
    """
    # 저장된 설정을 가져옵니다.
    with open(os.path.join(path, "config.json"), "r") as f:
        saved_configs = json.load(f)

    # 저장된 설정을 SavedConfig 객체로 변환합니다.
    saved_configs = SavedConfig(saved_configs)
    
    if override_backbone is not None:
        # 백본을 덮어쓸 경우 설정을 업데이트합니다.
        saved_configs.architecture = {"backbone": override_backbone,
                             "params": {}}
    # 네트워크 생성
    network = CustomLLM(saved_configs, saved_configs.eos_token_id)
    # 훈련된 가중치를 로드합니다.
    state_dict = torch.load(os.path.join(path, f"{model_name}.pt"))

    network.load_state_dict(state_dict)  # 네트워크에 가중치를 로드합니다.

    # 모델 생성
    clf = AbstractBaseModel(network=network,
                            mixed_precision=saved_configs.mixed_precision)
    clf.network.eval()  # 모델을 평가 모드로 설정합니다.
    return saved_configs, clf  # 설정과 모델을 반환합니다.

In [ ]:
MODEL_PATH = "/kaggle/input/simple-model-training/2/"  # 사전 훈련된 모델이 저장된 경로
MODEL_NAME = "fold_0"  # 로드할 모델의 이름

if INFERENCE:
    # 저장된 설정과 모델을 로드합니다.
    # 내 모델은 로컬 머신에서 MAX_LENGTH=1024로 훈련되었습니다.
    # Kaggle 노트북에서 훈련하지 않았기 때문에 백본을 덮어써야 합니다.
    saved_configs, saved_model = load_model(MODEL_PATH, MODEL_NAME, override_backbone="/kaggle/input/gemma/transformers/1.1-2b-it/1")
    # 메모리 사용을 제한하기 위해 배치 크기를 1로 설정합니다.
    saved_configs.batch_size = 1
    
    df_test = pd.read_csv(os.path.join(PATH_TO_DATA, "test.csv"))  # 테스트 데이터를 로드합니다.
    # train.csv와 일치하도록 더미 'score' 열을 생성합니다.
    df_test["score"] = -1
    # 테스트 데이터셋과 데이터로더를 생성합니다.
    ds_test, dl_test = get_dataset_and_loader(df_test, saved_configs, inference=True)
    
    test_preds = saved_model.predict_proba(dl_test).numpy()  # 예측 확률을 계산합니다.
    df_sub = pd.DataFrame()
    df_sub["essay_id"] = df_test["essay_id"]
    # 예측을 정수로 변환합니다.
    df_sub["score"] = np.rint(test_preds).astype(int)
    # 제출 파일을 저장합니다.
    df_sub.to_csv("submission.csv", index=None)